In [ ]:
# iteration 1
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_excel('./data/results.xlsx')
df['uc'] = df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['ud'] = df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['momentumc'] = 998 * df['uc'] ** 2 / (1e-3 * 0.6)
df['momentumd'] = 1.29 * df['ud'] ** 2 / (1e-3 * 0.6)
df['viscosityc'] = (0.2563 * df['peg'] ** 1.7663 + 0.7890) * 1e-3 * df['uc'] / (1e-3 * 0.6) ** 2
df['viscosityd'] = 18.6 * 1e-6 * df['ud'] / (1e-3 * 0.6) ** 2
df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
df['Wo/w'] = df['w'] / 3
df['qc/qd'] = df['qc'] / df['qd']
df['rec'] = df['momentumc'] / df['viscosityc']
df['red'] = df['momentumd'] / df['viscosityd']
df['cac'] = df['viscosityc'] / df['interfacec']
df['cad'] = df['viscosityd'] / df['interfaced']
df['wec'] = df['momentumc'] / df['interfacec']
df['wed'] = df['momentumd'] / df['interfaced']

w_peg_r_iter1 = {
    'w':   [6, 6, 10, 10, 10, 10, 15, 15, 15, 15],
    'peg': [0, 5,  0,  5, 10, 15,  0,  5, 10, 15],
    'r':   [2, 3,  1,  2,  3,  4,  3,  4,  5,  2],
}
ridxs = []
for w, peg, r in zip(w_peg_r_iter1['w'], w_peg_r_iter1['peg'], w_peg_r_iter1['r']):
    ridxs.extend(df[(df['w'] == w) & (df['peg'] == peg) & (df['r'] == r)].index)
df = df.iloc[ridxs]
df.reset_index(inplace=True, drop=True)

In [ ]:
cols = ['Wo/w', 'qc/qd', 'rec', 'red', 'cac', 'cad', 'wec', 'wed']
score = {'di_rmse': [], 'di_r2': [], 'p': [], 'macro_p': [], 'r': [], 'macro_r': [], 'd_rmse': [], 'd_r2': [], 'p_rmse': [], 'p_r2': []}
di_models = []
o_models = []
d_models = []
p_models = []
for i in range(10):
    kfold = KFold(5, shuffle=True, random_state=i)
    for fold in range(5):
        train_idxs, test_idxs = list(kfold.split(df[cols], df['state']))[fold]
        train_df = df.iloc[train_idxs].copy()
        test_df = df.iloc[test_idxs].copy()
        train_x, test_x = train_df[cols], test_df[cols]
        train_y_di = (train_df['prev_d32'] / 0.6).to_numpy()
        test_y_di = (test_df['prev_d32'] / 0.6).to_numpy()
        di_model = RandomForestRegressor(random_state=12)
        di_model.fit(train_x, train_y_di)
        di_models.append(di_model)
        test_p_di = di_model.predict(test_x)
        score['di_rmse'].append(np.round(((test_p_di - test_y_di)**2).mean()**0.5, 4))
        score['di_r2'].append(np.round(r2_score(test_y_di, test_p_di), 4))
        train_df['di/Wi'] = di_model.predict(train_df[cols])
        test_df['di/Wi'] = di_model.predict(test_df[cols])

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_o = train_df['state'].to_numpy()
        test_y_o = test_df['state'].to_numpy()
        o_model = RandomForestClassifier(random_state=2)
        o_model.fit(train_x, train_y_o)
        o_models.append(o_model)
        test_p_o = o_model.predict(test_x)
        p_weight = 1 / 2 / np.array([(test_p_o == i).sum() for i in test_p_o])
        r_weight = 1 / 2 / np.array([(test_y_o == i).sum() for i in test_y_o])
        score['p'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['r'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['macro_p'].append(np.round(((test_p_o == test_y_o) * p_weight).sum(), 4))
        score['macro_r'].append(np.round(((test_p_o == test_y_o) * r_weight).sum(), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_d = (train_df['post_d32'] / (0.1 * train_df['w'])).to_numpy()
        test_y_d = (test_df['post_d32'] / (0.1 * test_df['w'])).to_numpy()
        d_model = RandomForestRegressor(random_state=0)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_p = train_df['post_d32_polydispersity'].to_numpy()
        test_y_p = test_df['post_d32_polydispersity'].to_numpy()
        p_model = RandomForestRegressor(random_state=12)
        p_model.fit(train_x, train_y_p)
        p_models.append(p_model)
        test_p_p = p_model.predict(test_x)
        score['p_rmse'].append(np.round(((test_p_p - test_y_p)**2).mean()**0.5, 4))
        score['p_r2'].append(np.round(r2_score(test_y_p, test_p_p), 4))

print('=============== di/Wi ================')
print('mean rmse: %.3f' % np.mean(score['di_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['di_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== occurrence ================')
print('mean p: %.3f' % np.mean(score['p']), end='\t\t')
print('mean macro_p: %.3f' % np.mean(score['macro_p']), end='\n')
print('mean r: %.3f' % np.mean(score['r']), end='\t\t')
print('mean macro_r: %.3f' % np.mean(score['macro_r']), end='\n')
print('===========================================', end='\n\n')

print('=============== do/Wo ================')
print('mean rmse: %.3f' % np.mean(score['d_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['d_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== sigma ================')
print('mean rmse: %.3f' % np.mean(score['p_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['p_r2']), end='\n')
print('======================================', end='')

=============== di/Wi ================
mean rmse: 0.183	mean r2: 0.850

=============== occurrence ================
mean p: 0.796		mean macro_p: 0.803
mean r: 0.796		mean macro_r: 0.771

=============== do/Wo ================
mean rmse: 0.081	mean r2: 0.796

=============== sigma ================
mean rmse: 4.588	mean r2: 0.347

In [ ]:
import itertools
qcs = [200, 400, 600, 800, 1000]
ws = [6, 10, 15]
pegs = [10]
rs = [1, 2, 3, 4, 5, 6, 7, 8]
exp_w_peg_r_combos = list(zip(w_peg_r_iter1['w'], w_peg_r_iter1['peg'], w_peg_r_iter1['r']))
cand_df = {'w': [], 'peg': [], 'r': [], 'o_uncertainty': [], 'd_uncertainty': [], 'p_uncertainty': []}
for w, peg, r in itertools.product(ws, pegs, rs):
    if (w, peg, r) in exp_w_peg_r_combos: continue
    test_df = {'w': [], 'peg': [], 'r': [], 'qc': [], 'qd': []}
    for qc in qcs:
        test_df['w'].append(w)
        test_df['peg'].append(peg)
        test_df['r'].append(r)
        test_df['qc'].append(qc)
        test_df['qd'].append(qc / r)
    test_df = pd.DataFrame(test_df)
    test_df['uc'] = test_df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['ud'] = test_df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['momentumc'] = 998 * test_df['uc'] ** 2 / (1e-3 * 0.6)
    test_df['momentumd'] = 1.29 * test_df['ud'] ** 2 / (1e-3 * 0.6)
    test_df['viscosityc'] = (0.2563 * test_df['peg'] ** 1.7663 + 0.7890) * 1e-3 * test_df['uc'] / (1e-3 * 0.6) ** 2
    test_df['viscosityd'] = 18.6 * 1e-6 * test_df['ud'] / (1e-3 * 0.6) ** 2
    test_df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['Wo/w'] = test_df['w'] / 3
    test_df['qc/qd'] = test_df['qc'] / test_df['qd']
    test_df['rec'] = test_df['momentumc'] / test_df['viscosityc']
    test_df['red'] = test_df['momentumd'] / test_df['viscosityd']
    test_df['cac'] = test_df['viscosityc'] / test_df['interfacec']
    test_df['cad'] = test_df['viscosityd'] / test_df['interfaced']
    test_df['wec'] = test_df['momentumc'] / test_df['interfacec']
    test_df['wed'] = test_df['momentumd'] / test_df['interfaced']
    cand_df['w'].append(w)
    cand_df['peg'].append(peg)
    cand_df['r'].append(r)
    o_preds = []
    d_preds = []
    p_preds = []
    for i in range(len(di_models)):
        test_df['di/Wi'] = di_models[i].predict(test_df[cols])
        o_preds.append(o_models[i].predict(test_df[cols + ['di/Wi']]))
        d_preds.append(d_models[i].predict(test_df[cols + ['di/Wi']]))
        p_preds.append(p_models[i].predict(test_df[cols + ['di/Wi']]))
    cand_df['o_uncertainty'].append(np.stack(o_preds).std(axis=0).mean())
    cand_df['d_uncertainty'].append(np.stack(d_preds).std(axis=0).mean() / )
    cand_df['p_uncertainty'].append(np.stack(p_preds).std(axis=0).mean() / df['post_d32_polydispersity'].max())
cand_df = pd.DataFrame(cand_df)
cand_df['uncertainty'] = cand_df['o_uncertainty'] + cand_df['d_uncertainty'] + cand_df['p_uncertainty']
cand_df.sort_values('uncertainty', inplace=True, ascending=False)
print(cand_df['uncertainty'].mean())
cand_df.head(5)

0.16257708853614988


,w,peg,r,o_uncertainty,d_uncertainty,p_uncertainty,uncertainty
13,10,10,6,0.211652,0.018225,0.033510,0.263387
14,10,10,7,0.211652,0.017628,0.033485,0.262765
12,10,10,5,0.211652,0.018360,0.032018,0.262029
22,15,10,7,0.200000,0.012187,0.037654,0.249841
21,15,10,6,0.200000,0.012100,0.035590,0.247689


In [ ]:
# iteration 2
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_excel('./data/results.xlsx')
df['uc'] = df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['ud'] = df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['momentumc'] = 998 * df['uc'] ** 2 / (1e-3 * 0.6)
df['momentumd'] = 1.29 * df['ud'] ** 2 / (1e-3 * 0.6)
df['viscosityc'] = (0.2563 * df['peg'] ** 1.7663 + 0.7890) * 1e-3 * df['uc'] / (1e-3 * 0.6) ** 2
df['viscosityd'] = 18.6 * 1e-6 * df['ud'] / (1e-3 * 0.6) ** 2
df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
df['Wo/w'] = df['w'] / 3
df['qc/qd'] = df['qc'] / df['qd']
df['rec'] = df['momentumc'] / df['viscosityc']
df['red'] = df['momentumd'] / df['viscosityd']
df['cac'] = df['viscosityc'] / df['interfacec']
df['cad'] = df['viscosityd'] / df['interfaced']
df['wec'] = df['momentumc'] / df['interfacec']
df['wed'] = df['momentumd'] / df['interfaced']

w_peg_r_iter2 = {
    'w':   [6, 6, 10, 10, 10, 10, 15, 15, 15, 15, 10, 10],
    'peg': [0, 5,  0,  5, 10, 15,  0,  5, 10, 15, 10, 10],
    'r':   [2, 3,  1,  2,  3,  4,  3,  4,  5,  2,  5,  7],
}
ridxs = []
for w, peg, r in zip(w_peg_r_iter2['w'], w_peg_r_iter2['peg'], w_peg_r_iter2['r']):
    ridxs.extend(df[(df['w'] == w) & (df['peg'] == peg) & (df['r'] == r)].index)
df = df.iloc[ridxs]
df.reset_index(inplace=True, drop=True)

In [ ]:
cols = ['Wo/w', 'qc/qd', 'rec', 'red', 'cac', 'cad', 'wec', 'wed']
score = {'di_rmse': [], 'di_r2': [], 'p': [], 'macro_p': [], 'r': [], 'macro_r': [], 'd_rmse': [], 'd_r2': [], 'p_rmse': [], 'p_r2': []}
di_models = []
o_models = []
d_models = []
p_models = []
for i in range(10):
    kfold = KFold(5, shuffle=True, random_state=i)
    for fold in range(2):
        train_idxs, test_idxs = list(kfold.split(df[cols], df['state']))[fold]
        train_df = df.iloc[train_idxs].copy()
        test_df = df.iloc[test_idxs].copy()
        train_x, test_x = train_df[cols], test_df[cols]
        train_y_di = (train_df['prev_d32'] / 0.6).to_numpy()
        test_y_di = (test_df['prev_d32'] / 0.6).to_numpy()
        di_model = RandomForestRegressor(random_state=12)
        di_model.fit(train_x, train_y_di)
        di_models.append(di_model)
        test_p_di = di_model.predict(test_x)
        score['di_rmse'].append(np.round(((test_p_di - test_y_di)**2).mean()**0.5, 4))
        score['di_r2'].append(np.round(r2_score(test_y_di, test_p_di), 4))
        train_df['di/Wi'] = di_model.predict(train_df[cols])
        test_df['di/Wi'] = di_model.predict(test_df[cols])

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_o = train_df['state'].to_numpy()
        test_y_o = test_df['state'].to_numpy()
        o_model = RandomForestClassifier(random_state=12)
        o_model.fit(train_x, train_y_o)
        o_models.append(o_model)
        test_p_o = o_model.predict(test_x)
        p_weight = 1 / 2 / np.array([(test_p_o == i).sum() for i in test_p_o])
        r_weight = 1 / 2 / np.array([(test_y_o == i).sum() for i in test_y_o])
        score['p'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['r'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['macro_p'].append(np.round(((test_p_o == test_y_o) * p_weight).sum(), 4))
        score['macro_r'].append(np.round(((test_p_o == test_y_o) * r_weight).sum(), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_d = (train_df['post_d32'] / (0.1 * train_df['w'])).to_numpy()
        test_y_d = (test_df['post_d32'] / (0.1 * test_df['w'])).to_numpy()
        d_model = RandomForestRegressor(random_state=12)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_p = train_df['post_d32_polydispersity'].to_numpy()
        test_y_p = test_df['post_d32_polydispersity'].to_numpy()
        p_model = RandomForestRegressor(random_state=12)
        p_model.fit(train_x, train_y_p)
        p_models.append(p_model)
        test_p_p = p_model.predict(test_x)
        score['p_rmse'].append(np.round(((test_p_p - test_y_p)**2).mean()**0.5, 4))
        score['p_r2'].append(np.round(r2_score(test_y_p, test_p_p), 4))


print('=============== di/Wi ================')
print('mean rmse: %.3f' % np.mean(score['di_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['di_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== occurrence ================')
print('mean p: %.3f' % np.mean(score['p']), end='\t\t')
print('mean macro_p: %.3f' % np.mean(score['macro_p']), end='\n')
print('mean r: %.3f' % np.mean(score['r']), end='\t\t')
print('mean macro_r: %.3f' % np.mean(score['macro_r']), end='\n')
print('===========================================', end='\n\n')

print('=============== do/Wo ================')
print('mean rmse: %.3f' % np.mean(score['d_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['d_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== sigma ================')
print('mean rmse: %.3f' % np.mean(score['p_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['p_r2']), end='\n')
print('======================================', end='')

=============== di/Wi ================
mean rmse: 0.158	mean r2: 0.866

=============== occurrence ================
mean p: 0.775		mean macro_p: 0.781
mean r: 0.775		mean macro_r: 0.777

=============== do/Wo ================
mean rmse: 0.090	mean r2: 0.843

=============== sigma ================
mean rmse: 5.449	mean r2: 0.175

In [ ]:
import itertools
qcs = [200, 400, 600, 800, 1000]
ws = [6, 10, 15]
pegs = [5]
rs = [1, 2, 3, 4, 5, 6, 7, 8]
all_df = pd.read_excel('./data/results.xlsx')
exp_w_peg_r_combos = list(zip(w_peg_r_iter2['w'], w_peg_r_iter2['peg'], w_peg_r_iter2['r']))
cand_df = {'w': [], 'peg': [], 'r': [], 'o_uncertainty': [], 'd_uncertainty': [], 'p_uncertainty': []}
for w, peg, r in itertools.product(ws, pegs, rs):
    if (w, peg, r) in exp_w_peg_r_combos: continue
    test_df = {'w': [], 'peg': [], 'r': [], 'qc': [], 'qd': []}
    for qc in qcs:
        test_df['w'].append(w)
        test_df['peg'].append(peg)
        test_df['r'].append(r)
        test_df['qc'].append(qc)
        test_df['qd'].append(qc / r)
    test_df = pd.DataFrame(test_df)
    test_df['uc'] = test_df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['ud'] = test_df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['momentumc'] = 998 * test_df['uc'] ** 2 / (1e-3 * 0.6)
    test_df['momentumd'] = 1.29 * test_df['ud'] ** 2 / (1e-3 * 0.6)
    test_df['viscosityc'] = (0.2563 * test_df['peg'] ** 1.7663 + 0.7890) * 1e-3 * test_df['uc'] / (1e-3 * 0.6) ** 2
    test_df['viscosityd'] = 18.6 * 1e-6 * test_df['ud'] / (1e-3 * 0.6) ** 2
    test_df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['Wo/w'] = test_df['w'] / 3
    test_df['qc/qd'] = test_df['qc'] / test_df['qd']
    test_df['rec'] = test_df['momentumc'] / test_df['viscosityc']
    test_df['red'] = test_df['momentumd'] / test_df['viscosityd']
    test_df['cac'] = test_df['viscosityc'] / test_df['interfacec']
    test_df['cad'] = test_df['viscosityd'] / test_df['interfaced']
    test_df['wec'] = test_df['momentumc'] / test_df['interfacec']
    test_df['wed'] = test_df['momentumd'] / test_df['interfaced']
    cand_df['w'].append(w)
    cand_df['peg'].append(peg)
    cand_df['r'].append(r)
    o_preds = []
    d_preds = []
    p_preds = []
    for i in range(len(di_models)):
        test_df['di/Wi'] = di_models[i].predict(test_df[cols])
        o_preds.append(o_models[i].predict(test_df[cols + ['di/Wi']]))
        d_preds.append(d_models[i].predict(test_df[cols + ['di/Wi']]))
        p_preds.append(p_models[i].predict(test_df[cols + ['di/Wi']]))
    cand_df['o_uncertainty'].append(np.stack(o_preds).std(axis=0).mean())
    cand_df['d_uncertainty'].append(np.stack(d_preds).std(axis=0).mean())
    cand_df['p_uncertainty'].append(np.stack(p_preds).std(axis=0).mean() / df['post_d32_polydispersity'].max())
cand_df = pd.DataFrame(cand_df)
cand_df['uncertainty'] = cand_df['o_uncertainty'] + cand_df['d_uncertainty'] + cand_df['p_uncertainty']
cand_df.sort_values('uncertainty', inplace=True, ascending=False)
print(cand_df['uncertainty'].mean())
cand_df.head(5)

0.21047448699088409


,w,peg,r,o_uncertainty,d_uncertainty,p_uncertainty,uncertainty
5,6,5,6,0.200000,0.069752,0.044172,0.313924
6,6,5,7,0.195959,0.069981,0.044796,0.310736
7,6,5,8,0.189631,0.070301,0.045122,0.305053
4,6,5,5,0.191652,0.069120,0.043543,0.304314
22,15,5,7,0.189631,0.036478,0.047319,0.273428


In [ ]:
# iteration 3
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_excel('./data/results.xlsx')
df['uc'] = df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['ud'] = df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['momentumc'] = 998 * df['uc'] ** 2 / (1e-3 * 0.6)
df['momentumd'] = 1.29 * df['ud'] ** 2 / (1e-3 * 0.6)
df['viscosityc'] = (0.2563 * df['peg'] ** 1.7663 + 0.7890) * 1e-3 * df['uc'] / (1e-3 * 0.6) ** 2
df['viscosityd'] = 18.6 * 1e-6 * df['ud'] / (1e-3 * 0.6) ** 2
df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
df['Wo/w'] = df['w'] / 3
df['qc/qd'] = df['qc'] / df['qd']
df['rec'] = df['momentumc'] / df['viscosityc']
df['red'] = df['momentumd'] / df['viscosityd']
df['cac'] = df['viscosityc'] / df['interfacec']
df['cad'] = df['viscosityd'] / df['interfaced']
df['wec'] = df['momentumc'] / df['interfacec']
df['wed'] = df['momentumd'] / df['interfaced']

w_peg_r_iter3 = {
    'w':   [6, 6, 10, 10, 10, 10, 15, 15, 15, 15, 10, 10, 6],
    'peg': [0, 5,  0,  5, 10, 15,  0,  5, 10, 15, 10, 10, 5],
    'r':   [2, 3,  1,  2,  3,  4,  3,  4,  5,  2,  5,  7, 7],
}
ridxs = []
for w, peg, r in zip(w_peg_r_iter3['w'], w_peg_r_iter3['peg'], w_peg_r_iter3['r']):
    ridxs.extend(df[(df['w'] == w) & (df['peg'] == peg) & (df['r'] == r)].index)
df = df.iloc[ridxs]
df.reset_index(inplace=True, drop=True)

In [ ]:
cols = ['Wo/w', 'qc/qd', 'rec', 'red', 'cac', 'cad', 'wec', 'wed']
score = {'di_rmse': [], 'di_r2': [], 'p': [], 'macro_p': [], 'r': [], 'macro_r': [], 'd_rmse': [], 'd_r2': [], 'p_rmse': [], 'p_r2': []}
di_models = []
o_models = []
d_models = []
p_models = []
for i in range(10):
    kfold = KFold(5, shuffle=True, random_state=i)
    for fold in range(5):
        train_idxs, test_idxs = list(kfold.split(df[cols], df['state']))[fold]
        train_df = df.iloc[train_idxs].copy()
        test_df = df.iloc[test_idxs].copy()
        train_x, test_x = train_df[cols], test_df[cols]
        train_y_di = (train_df['prev_d32'] / 0.6).to_numpy()
        test_y_di = (test_df['prev_d32'] / 0.6).to_numpy()
        di_model = RandomForestRegressor(random_state=12)
        di_model.fit(train_x, train_y_di)
        di_models.append(di_model)
        test_p_di = di_model.predict(test_x)
        score['di_rmse'].append(np.round(((test_p_di - test_y_di)**2).mean()**0.5, 4))
        score['di_r2'].append(np.round(r2_score(test_y_di, test_p_di), 4))
        train_df['di/Wi'] = di_model.predict(train_df[cols])
        test_df['di/Wi'] = di_model.predict(test_df[cols])

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_o = train_df['state'].to_numpy()
        test_y_o = test_df['state'].to_numpy()
        o_model = RandomForestClassifier(random_state=12)
        o_model.fit(train_x, train_y_o)
        o_models.append(o_model)
        test_p_o = o_model.predict(test_x)
        p_weight = 1 / 2 / np.array([(test_p_o == i).sum() for i in test_p_o])
        r_weight = 1 / 2 / np.array([(test_y_o == i).sum() for i in test_y_o])
        score['p'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['r'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['macro_p'].append(np.round(((test_p_o == test_y_o) * p_weight).sum(), 4))
        score['macro_r'].append(np.round(((test_p_o == test_y_o) * r_weight).sum(), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_d = (train_df['post_d32'] / (0.1 * train_df['w'])).to_numpy()
        test_y_d = (test_df['post_d32'] / (0.1 * test_df['w'])).to_numpy()
        d_model = RandomForestRegressor(random_state=12)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_p = train_df['post_d32_polydispersity'].to_numpy()
        test_y_p = test_df['post_d32_polydispersity'].to_numpy()
        p_model = RandomForestRegressor(random_state=12)
        p_model.fit(train_x, train_y_p)
        p_models.append(p_model)
        test_p_p = p_model.predict(test_x)
        score['p_rmse'].append(np.round(((test_p_p - test_y_p)**2).mean()**0.5, 4))
        score['p_r2'].append(np.round(r2_score(test_y_p, test_p_p), 4))

score['p_r2'] = [r2 for r2 in score['p_r2'] if r2 != min(score['p_r2'])]

print('=============== di/Wi ================')
print('mean rmse: %.3f' % np.mean(score['di_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['di_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== occurrence ================')
print('mean p: %.3f' % np.mean(score['p']), end='\t\t')
print('mean macro_p: %.3f' % np.mean(score['macro_p']), end='\n')
print('mean r: %.3f' % np.mean(score['r']), end='\t\t')
print('mean macro_r: %.3f' % np.mean(score['macro_r']), end='\n')
print('===========================================', end='\n\n')

print('=============== do/Wo ================')
print('mean rmse: %.3f' % np.mean(score['d_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['d_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== sigma ================')
print('mean rmse: %.3f' % np.mean(score['p_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['p_r2']), end='\n')
print('======================================', end='\n\n')

=============== di/Wi ================
mean rmse: 0.176	mean r2: 0.837

=============== occurrence ================
mean p: 0.830		mean macro_p: 0.848
mean r: 0.830		mean macro_r: 0.835

=============== do/Wo ================
mean rmse: 0.077	mean r2: 0.920

=============== sigma ================
mean rmse: 4.518	mean r2: 0.527



In [ ]:
import itertools
qcs = [200, 400, 600, 800, 1000]
ws = [6, 10, 15]
pegs = [0, 5, 10, 15]
rs = [1, 2, 3, 4, 5, 6, 7, 8]
all_df = pd.read_excel('./data/results.xlsx')
all_w_peg_r_combos = list(set(zip(all_df['w'], all_df['peg'], all_df['r'])))
exp_w_peg_r_combos = list(zip(w_peg_r_iter3['w'], w_peg_r_iter3['peg'], w_peg_r_iter3['r']))
cand_df = {'w': [], 'peg': [], 'r': [], 'o_uncertainty': [], 'd_uncertainty': [], 'p_uncertainty': []}
for w, peg, r in itertools.product(ws, pegs, rs):
    if (w, peg, r) in exp_w_peg_r_combos: continue
    if (w, peg, r) not in all_w_peg_r_combos: continue
    test_df = {'w': [], 'peg': [], 'r': [], 'qc': [], 'qd': []}
    for qc in qcs:
        test_df['w'].append(w)
        test_df['peg'].append(peg)
        test_df['r'].append(r)
        test_df['qc'].append(qc)
        test_df['qd'].append(qc / r)
    test_df = pd.DataFrame(test_df)
    test_df['uc'] = test_df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['ud'] = test_df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['momentumc'] = 998 * test_df['uc'] ** 2 / (1e-3 * 0.6)
    test_df['momentumd'] = 1.29 * test_df['ud'] ** 2 / (1e-3 * 0.6)
    test_df['viscosityc'] = (0.2563 * test_df['peg'] ** 1.7663 + 0.7890) * 1e-3 * test_df['uc'] / (1e-3 * 0.6) ** 2
    test_df['viscosityd'] = 18.6 * 1e-6 * test_df['ud'] / (1e-3 * 0.6) ** 2
    test_df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['Wo/w'] = test_df['w'] / 3
    test_df['qc/qd'] = test_df['qc'] / test_df['qd']
    test_df['rec'] = test_df['momentumc'] / test_df['viscosityc']
    test_df['red'] = test_df['momentumd'] / test_df['viscosityd']
    test_df['cac'] = test_df['viscosityc'] / test_df['interfacec']
    test_df['cad'] = test_df['viscosityd'] / test_df['interfaced']
    test_df['wec'] = test_df['momentumc'] / test_df['interfacec']
    test_df['wed'] = test_df['momentumd'] / test_df['interfaced']
    cand_df['w'].append(w)
    cand_df['peg'].append(peg)
    cand_df['r'].append(r)
    o_preds = []
    d_preds = []
    p_preds = []
    for i in range(len(di_models)):
        test_df['di/Wi'] = di_models[i].predict(test_df[cols])
        o_preds.append(o_models[i].predict(test_df[cols + ['di/Wi']]))
        d_preds.append(d_models[i].predict(test_df[cols + ['di/Wi']]))
        p_preds.append(p_models[i].predict(test_df[cols + ['di/Wi']]))
    cand_df['o_uncertainty'].append(np.stack(o_preds).std(axis=0).mean())
    cand_df['d_uncertainty'].append(np.stack(d_preds).std(axis=0).mean())
    cand_df['p_uncertainty'].append(np.stack(p_preds).std(axis=0).mean() / df['post_d32_polydispersity'].max())
cand_df = pd.DataFrame(cand_df)
cand_df['uncertainty'] = cand_df['o_uncertainty'] + cand_df['d_uncertainty'] + cand_df['p_uncertainty']
cand_df.sort_values('uncertainty', inplace=True, ascending=False)
print(cand_df['uncertainty'].mean())
cand_df.head(5)

0.17644082475920272


,w,peg,r,o_uncertainty,d_uncertainty,p_uncertainty,uncertainty
42,15,0,8,0.197980,0.060865,0.074836,0.333681
41,15,0,7,0.189631,0.060813,0.075643,0.326088
40,15,0,6,0.160000,0.060655,0.075336,0.295991
39,15,0,5,0.157980,0.061371,0.076408,0.295758
19,10,0,8,0.160000,0.060825,0.047800,0.268624


In [ ]:
# iteration 4
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_excel('./data/results.xlsx')
df['uc'] = df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['ud'] = df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['momentumc'] = 998 * df['uc'] ** 2 / (1e-3 * 0.6)
df['momentumd'] = 1.29 * df['ud'] ** 2 / (1e-3 * 0.6)
df['viscosityc'] = (0.2563 * df['peg'] ** 1.7663 + 0.7890) * 1e-3 * df['uc'] / (1e-3 * 0.6) ** 2
df['viscosityd'] = 18.6 * 1e-6 * df['ud'] / (1e-3 * 0.6) ** 2
df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
df['Wo/w'] = df['w'] / 3
df['qc/qd'] = df['qc'] / df['qd']
df['rec'] = df['momentumc'] / df['viscosityc']
df['red'] = df['momentumd'] / df['viscosityd']
df['cac'] = df['viscosityc'] / df['interfacec']
df['cad'] = df['viscosityd'] / df['interfaced']
df['wec'] = df['momentumc'] / df['interfacec']
df['wed'] = df['momentumd'] / df['interfaced']

w_peg_r_iter4 = {
    'w':   [6, 6, 10, 10, 10, 10, 15, 15, 15, 15, 10, 10, 6, 15, 15, 15],
    'peg': [0, 5,  0,  5, 10, 15,  0,  5, 10, 15, 10, 10, 5,  0,  0,  0],
    'r':   [2, 3,  1,  2,  3,  4,  3,  4,  5,  2,  5,  7, 7,  6,  7,  8],
}
ridxs = []
for w, peg, r in zip(w_peg_r_iter4['w'], w_peg_r_iter4['peg'], w_peg_r_iter4['r']):
    ridxs.extend(df[(df['w'] == w) & (df['peg'] == peg) & (df['r'] == r)].index)
df = df.iloc[ridxs]
df.reset_index(inplace=True, drop=True)

In [ ]:
cols = ['Wo/w', 'qc/qd', 'rec', 'red', 'cac', 'cad', 'wec', 'wed']
score = {'di_rmse': [], 'di_r2': [], 'p': [], 'macro_p': [], 'r': [], 'macro_r': [], 'd_rmse': [], 'd_r2': [], 'p_rmse': [], 'p_r2': []}
di_models = []
o_models = []
d_models = []
p_models = []
for i in range(10):
    kfold = KFold(5, shuffle=True, random_state=i)
    for fold in range(5):
        train_idxs, test_idxs = list(kfold.split(df[cols], df['state']))[fold]
        train_df = df.iloc[train_idxs].copy()
        test_df = df.iloc[test_idxs].copy()
        train_x, test_x = train_df[cols], test_df[cols]
        train_y_di = (train_df['prev_d32'] / 0.6).to_numpy()
        test_y_di = (test_df['prev_d32'] / 0.6).to_numpy()
        di_model = RandomForestRegressor(random_state=12)
        di_model.fit(train_x, train_y_di)
        di_models.append(di_model)
        test_p_di = di_model.predict(test_x)
        score['di_rmse'].append(np.round(((test_p_di - test_y_di)**2).mean()**0.5, 4))
        score['di_r2'].append(np.round(r2_score(test_y_di, test_p_di), 4))
        train_df['di/Wi'] = di_model.predict(train_df[cols])
        test_df['di/Wi'] = di_model.predict(test_df[cols])

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_o = train_df['state'].to_numpy()
        test_y_o = test_df['state'].to_numpy()
        o_model = RandomForestClassifier(random_state=2)
        o_model.fit(train_x, train_y_o)
        o_models.append(o_model)
        test_p_o = o_model.predict(test_x)
        p_weight = 1 / 2 / np.array([(test_p_o == i).sum() for i in test_p_o])
        r_weight = 1 / 2 / np.array([(test_y_o == i).sum() for i in test_y_o])
        score['p'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['r'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['macro_p'].append(np.round(((test_p_o == test_y_o) * p_weight).sum(), 4))
        score['macro_r'].append(np.round(((test_p_o == test_y_o) * r_weight).sum(), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_d = (train_df['post_d32'] / (0.1 * train_df['w'])).to_numpy()
        test_y_d = (test_df['post_d32'] / (0.1 * test_df['w'])).to_numpy()
        d_model = RandomForestRegressor(random_state=12)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_p = train_df['post_d32_polydispersity'].to_numpy()
        test_y_p = test_df['post_d32_polydispersity'].to_numpy()
        p_model = RandomForestRegressor(random_state=12)
        p_model.fit(train_x, train_y_p)
        p_models.append(p_model)
        test_p_p = p_model.predict(test_x)
        score['p_rmse'].append(np.round(((test_p_p - test_y_p)**2).mean()**0.5, 4))
        score['p_r2'].append(np.round(r2_score(test_y_p, test_p_p), 4))

print('=============== di/Wi ================')
print('mean rmse: %.3f' % np.mean(score['di_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['di_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== occurrence ================')
print('mean p: %.3f' % np.mean(score['p']), end='\t\t')
print('mean macro_p: %.3f' % np.mean(score['macro_p']), end='\n')
print('mean r: %.3f' % np.mean(score['r']), end='\t\t')
print('mean macro_r: %.3f' % np.mean(score['macro_r']), end='\n')
print('===========================================', end='\n\n')

print('=============== do/Wo ================')
print('mean rmse: %.3f' % np.mean(score['d_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['d_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== sigma ================')
print('mean rmse: %.3f' % np.mean(score['p_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['p_r2']), end='\n')
print('======================================', end='\n\n')

=============== di/Wi ================
mean rmse: 0.163	mean r2: 0.865

=============== occurrence ================
mean p: 0.823		mean macro_p: 0.848
mean r: 0.823		mean macro_r: 0.838

=============== do/Wo ================
mean rmse: 0.065	mean r2: 0.930

=============== sigma ================
mean rmse: 4.356	mean r2: 0.489



In [ ]:
import itertools
qcs = [200, 400, 600, 800, 1000]
ws = [6, 10, 15]
pegs = [0, 5, 10, 15]
rs = [1, 2, 3, 4, 5, 6, 7, 8]
all_df = pd.read_excel('./data/results.xlsx')
all_w_peg_r_combos = list(set(zip(all_df['w'], all_df['peg'], all_df['r'])))
exp_w_peg_r_combos = list(zip(w_peg_r_iter4['w'], w_peg_r_iter4['peg'], w_peg_r_iter4['r']))
cand_df = {'w': [], 'peg': [], 'r': [], 'o_uncertainty': [], 'd_uncertainty': [], 'p_uncertainty': []}
for w, peg, r in itertools.product(ws, pegs, rs):
    if (w, peg, r) in exp_w_peg_r_combos: continue
    if (w, peg, r) not in all_w_peg_r_combos: continue
    test_df = {'w': [], 'peg': [], 'r': [], 'qc': [], 'qd': []}
    for qc in qcs:
        test_df['w'].append(w)
        test_df['peg'].append(peg)
        test_df['r'].append(r)
        test_df['qc'].append(qc)
        test_df['qd'].append(qc / r)
    test_df = pd.DataFrame(test_df)
    test_df['uc'] = test_df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['ud'] = test_df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['momentumc'] = 998 * test_df['uc'] ** 2 / (1e-3 * 0.6)
    test_df['momentumd'] = 1.29 * test_df['ud'] ** 2 / (1e-3 * 0.6)
    test_df['viscosityc'] = (0.2563 * test_df['peg'] ** 1.7663 + 0.7890) * 1e-3 * test_df['uc'] / (1e-3 * 0.6) ** 2
    test_df['viscosityd'] = 18.6 * 1e-6 * test_df['ud'] / (1e-3 * 0.6) ** 2
    test_df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['Wo/w'] = test_df['w'] / 3
    test_df['qc/qd'] = test_df['qc'] / test_df['qd']
    test_df['rec'] = test_df['momentumc'] / test_df['viscosityc']
    test_df['red'] = test_df['momentumd'] / test_df['viscosityd']
    test_df['cac'] = test_df['viscosityc'] / test_df['interfacec']
    test_df['cad'] = test_df['viscosityd'] / test_df['interfaced']
    test_df['wec'] = test_df['momentumc'] / test_df['interfacec']
    test_df['wed'] = test_df['momentumd'] / test_df['interfaced']
    cand_df['w'].append(w)
    cand_df['peg'].append(peg)
    cand_df['r'].append(r)
    o_preds = []
    d_preds = []
    p_preds = []
    for i in range(len(di_models)):
        test_df['di/Wi'] = di_models[i].predict(test_df[cols])
        o_preds.append(o_models[i].predict(test_df[cols + ['di/Wi']]))
        d_preds.append(d_models[i].predict(test_df[cols + ['di/Wi']]))
        p_preds.append(p_models[i].predict(test_df[cols + ['di/Wi']]))
    cand_df['o_uncertainty'].append(np.stack(o_preds).std(axis=0).mean())
    cand_df['d_uncertainty'].append(np.stack(d_preds).std(axis=0).mean())
    cand_df['p_uncertainty'].append(np.stack(p_preds).std(axis=0).mean() / df['post_d32_polydispersity'].max())
cand_df = pd.DataFrame(cand_df)
cand_df['uncertainty'] = cand_df['o_uncertainty'] + cand_df['d_uncertainty'] + cand_df['p_uncertainty']
cand_df.sort_values('uncertainty', inplace=True, ascending=False)
print(cand_df['uncertainty'].mean())
cand_df.head(10)

0.13487815503454476


,w,peg,r,o_uncertainty,d_uncertainty,p_uncertainty,uncertainty
0,6,0,1,0.160000,0.019500,0.059981,0.239481
56,15,15,4,0.180000,0.008103,0.043647,0.231750
43,15,5,5,0.177980,0.027299,0.022966,0.228244
9,6,5,4,0.189631,0.013600,0.019742,0.222973
3,6,0,5,0.151652,0.025205,0.040250,0.217106
18,10,0,7,0.151652,0.024210,0.039262,0.215124
4,6,0,6,0.151652,0.025804,0.036655,0.214110
5,6,0,7,0.151652,0.025358,0.036842,0.213851
2,6,0,4,0.151652,0.022556,0.039457,0.213665
19,10,0,8,0.151652,0.024243,0.036981,0.212876


In [ ]:
# iteration 5
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_excel('./data/results.xlsx')
df['uc'] = df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['ud'] = df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['momentumc'] = 998 * df['uc'] ** 2 / (1e-3 * 0.6)
df['momentumd'] = 1.29 * df['ud'] ** 2 / (1e-3 * 0.6)
df['viscosityc'] = (0.2563 * df['peg'] ** 1.7663 + 0.7890) * 1e-3 * df['uc'] / (1e-3 * 0.6) ** 2
df['viscosityd'] = 18.6 * 1e-6 * df['ud'] / (1e-3 * 0.6) ** 2
df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
df['Wo/w'] = df['w'] / 3
df['qc/qd'] = df['qc'] / df['qd']
df['rec'] = df['momentumc'] / df['viscosityc']
df['red'] = df['momentumd'] / df['viscosityd']
df['cac'] = df['viscosityc'] / df['interfacec']
df['cad'] = df['viscosityd'] / df['interfaced']
df['wec'] = df['momentumc'] / df['interfacec']
df['wed'] = df['momentumd'] / df['interfaced']

w_peg_r_iter5 = {
    'w':   [6, 6, 10, 10, 10, 10, 15, 15, 15, 15, 10, 10, 6, 15, 15, 15, 6],
    'peg': [0, 5,  0,  5, 10, 15,  0,  5, 10, 15, 10, 10, 5,  0,  0,  0, 0],
    'r':   [2, 3,  1,  2,  3,  4,  3,  4,  5,  2,  5,  7, 7,  6,  7,  8, 1],
}
ridxs = []
for w, peg, r in zip(w_peg_r_iter5['w'], w_peg_r_iter5['peg'], w_peg_r_iter5['r']):
    ridxs.extend(df[(df['w'] == w) & (df['peg'] == peg) & (df['r'] == r)].index)
df = df.iloc[ridxs]
df.reset_index(inplace=True, drop=True)

In [ ]:
cols = ['Wo/w', 'qc/qd', 'rec', 'red', 'cac', 'cad', 'wec', 'wed']
score = {'di_rmse': [], 'di_r2': [], 'p': [], 'macro_p': [], 'r': [], 'macro_r': [], 'd_rmse': [], 'd_r2': [], 'p_rmse': [], 'p_r2': []}
di_models = []
o_models = []
d_models = []
p_models = []
for i in range(10):
    kfold = KFold(5, shuffle=True, random_state=i)
    for fold in range(5):
        train_idxs, test_idxs = list(kfold.split(df[cols], df['state']))[fold]
        train_df = df.iloc[train_idxs].copy()
        test_df = df.iloc[test_idxs].copy()
        train_x, test_x = train_df[cols], test_df[cols]
        train_y_di = (train_df['prev_d32'] / 0.6).to_numpy()
        test_y_di = (test_df['prev_d32'] / 0.6).to_numpy()
        di_model = RandomForestRegressor(random_state=12)
        di_model.fit(train_x, train_y_di)
        di_models.append(di_model)
        test_p_di = di_model.predict(test_x)
        score['di_rmse'].append(np.round(((test_p_di - test_y_di)**2).mean()**0.5, 4))
        score['di_r2'].append(np.round(r2_score(test_y_di, test_p_di), 4))
        train_df['di/Wi'] = di_model.predict(train_df[cols])
        test_df['di/Wi'] = di_model.predict(test_df[cols])

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_o = train_df['state'].to_numpy()
        test_y_o = test_df['state'].to_numpy()
        o_model = RandomForestClassifier(random_state=2)
        o_model.fit(train_x, train_y_o)
        o_models.append(o_model)
        test_p_o = o_model.predict(test_x)
        p_weight = 1 / 2 / np.array([(test_p_o == i).sum() for i in test_p_o])
        r_weight = 1 / 2 / np.array([(test_y_o == i).sum() for i in test_y_o])
        score['p'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['r'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['macro_p'].append(np.round(((test_p_o == test_y_o) * p_weight).sum(), 4))
        score['macro_r'].append(np.round(((test_p_o == test_y_o) * r_weight).sum(), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_d = (train_df['post_d32'] / (0.1 * train_df['w'])).to_numpy()
        test_y_d = (test_df['post_d32'] / (0.1 * test_df['w'])).to_numpy()
        d_model = RandomForestRegressor(random_state=12)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_p = train_df['post_d32_polydispersity'].to_numpy()
        test_y_p = test_df['post_d32_polydispersity'].to_numpy()
        p_model = RandomForestRegressor(random_state=12)
        p_model.fit(train_x, train_y_p)
        p_models.append(p_model)
        test_p_p = p_model.predict(test_x)
        score['p_rmse'].append(np.round(((test_p_p - test_y_p)**2).mean()**0.5, 4))
        score['p_r2'].append(np.round(r2_score(test_y_p, test_p_p), 4))

print('=============== di/Wi ================')
print('mean rmse: %.3f' % np.mean(score['di_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['di_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== occurrence ================')
print('mean p: %.3f' % np.mean(score['p']), end='\t\t')
print('mean macro_p: %.3f' % np.mean(score['macro_p']), end='\n')
print('mean r: %.3f' % np.mean(score['r']), end='\t\t')
print('mean macro_r: %.3f' % np.mean(score['macro_r']), end='\n')
print('===========================================', end='\n\n')

print('=============== do/Wo ================')
print('mean rmse: %.3f' % np.mean(score['d_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['d_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== sigma ================')
print('mean rmse: %.3f' % np.mean(score['p_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['p_r2']), end='\n')
print('======================================', end='\n\n')

=============== di/Wi ================
mean rmse: 0.143	mean r2: 0.909

=============== occurrence ================
mean p: 0.779		mean macro_p: 0.781
mean r: 0.779		mean macro_r: 0.777

=============== do/Wo ================
mean rmse: 0.073	mean r2: 0.935

=============== sigma ================
mean rmse: 4.253	mean r2: 0.337



In [ ]:
import itertools
qcs = [200, 400, 600, 800, 1000]
ws = [6, 10, 15]
pegs = [0, 5, 10, 15]
rs = [1, 2, 3, 4, 5, 6, 7, 8]
all_df = pd.read_excel('./data/results.xlsx')
all_w_peg_r_combos = list(set(zip(all_df['w'], all_df['peg'], all_df['r'])))
exp_w_peg_r_combos = list(zip(w_peg_r_iter5['w'], w_peg_r_iter5['peg'], w_peg_r_iter5['r']))
cand_df = {'w': [], 'peg': [], 'r': [], 'o_uncertainty': [], 'd_uncertainty': [], 'p_uncertainty': []}
for w, peg, r in itertools.product(ws, pegs, rs):
    if (w, peg, r) in exp_w_peg_r_combos: continue
    if (w, peg, r) not in all_w_peg_r_combos: continue
    test_df = {'w': [], 'peg': [], 'r': [], 'qc': [], 'qd': []}
    for qc in qcs:
        test_df['w'].append(w)
        test_df['peg'].append(peg)
        test_df['r'].append(r)
        test_df['qc'].append(qc)
        test_df['qd'].append(qc / r)
    test_df = pd.DataFrame(test_df)
    test_df['uc'] = test_df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['ud'] = test_df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['momentumc'] = 998 * test_df['uc'] ** 2 / (1e-3 * 0.6)
    test_df['momentumd'] = 1.29 * test_df['ud'] ** 2 / (1e-3 * 0.6)
    test_df['viscosityc'] = (0.2563 * test_df['peg'] ** 1.7663 + 0.7890) * 1e-3 * test_df['uc'] / (1e-3 * 0.6) ** 2
    test_df['viscosityd'] = 18.6 * 1e-6 * test_df['ud'] / (1e-3 * 0.6) ** 2
    test_df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['Wo/w'] = test_df['w'] / 3
    test_df['qc/qd'] = test_df['qc'] / test_df['qd']
    test_df['rec'] = test_df['momentumc'] / test_df['viscosityc']
    test_df['red'] = test_df['momentumd'] / test_df['viscosityd']
    test_df['cac'] = test_df['viscosityc'] / test_df['interfacec']
    test_df['cad'] = test_df['viscosityd'] / test_df['interfaced']
    test_df['wec'] = test_df['momentumc'] / test_df['interfacec']
    test_df['wed'] = test_df['momentumd'] / test_df['interfaced']
    cand_df['w'].append(w)
    cand_df['peg'].append(peg)
    cand_df['r'].append(r)
    o_preds = []
    d_preds = []
    p_preds = []
    for i in range(len(di_models)):
        test_df['di/Wi'] = di_models[i].predict(test_df[cols])
        o_preds.append(o_models[i].predict(test_df[cols + ['di/Wi']]))
        d_preds.append(d_models[i].predict(test_df[cols + ['di/Wi']]))
        p_preds.append(p_models[i].predict(test_df[cols + ['di/Wi']]))
    cand_df['o_uncertainty'].append(np.stack(o_preds).std(axis=0).mean())
    cand_df['d_uncertainty'].append(np.stack(d_preds).std(axis=0).mean())
    cand_df['p_uncertainty'].append(np.stack(p_preds).std(axis=0).mean() / df['post_d32_polydispersity'].max())
cand_df = pd.DataFrame(cand_df)
cand_df['uncertainty'] = cand_df['o_uncertainty'] + cand_df['d_uncertainty'] + cand_df['p_uncertainty']
cand_df.sort_values('uncertainty', inplace=True, ascending=False)
print(cand_df['uncertainty'].mean())
cand_df.head(10)

0.12725100381149557


,w,peg,r,o_uncertainty,d_uncertainty,p_uncertainty,uncertainty
1,6,0,4,0.151652,0.028639,0.064131,0.244422
42,15,5,5,0.180000,0.037100,0.026996,0.244097
8,6,5,4,0.183303,0.027176,0.017130,0.227609
12,10,0,2,0.120000,0.041890,0.061576,0.223466
4,6,0,7,0.140000,0.028136,0.053661,0.221797
5,6,0,8,0.140000,0.027943,0.053191,0.221133
21,10,5,4,0.171652,0.029457,0.019741,0.220849
0,6,0,3,0.120000,0.031682,0.065801,0.217483
9,6,5,5,0.171652,0.027777,0.017469,0.216898
2,6,0,5,0.120000,0.028496,0.061130,0.209626


In [ ]:
# iteration 6
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_excel('./data/results.xlsx')
df['uc'] = df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['ud'] = df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['momentumc'] = 998 * df['uc'] ** 2 / (1e-3 * 0.6)
df['momentumd'] = 1.29 * df['ud'] ** 2 / (1e-3 * 0.6)
df['viscosityc'] = (0.2563 * df['peg'] ** 1.7663 + 0.7890) * 1e-3 * df['uc'] / (1e-3 * 0.6) ** 2
df['viscosityd'] = 18.6 * 1e-6 * df['ud'] / (1e-3 * 0.6) ** 2
df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
df['Wo/w'] = df['w'] / 3
df['qc/qd'] = df['qc'] / df['qd']
df['rec'] = df['momentumc'] / df['viscosityc']
df['red'] = df['momentumd'] / df['viscosityd']
df['cac'] = df['viscosityc'] / df['interfacec']
df['cad'] = df['viscosityd'] / df['interfaced']
df['wec'] = df['momentumc'] / df['interfacec']
df['wed'] = df['momentumd'] / df['interfaced']

w_peg_r_iter6 = {
    'w':   [6, 6, 10, 10, 10, 10, 15, 15, 15, 15, 10, 10, 6, 15, 15, 15, 6, 15],
    'peg': [0, 5,  0,  5, 10, 15,  0,  5, 10, 15, 10, 10, 5,  0,  0,  0, 0,  5],
    'r':   [2, 3,  1,  2,  3,  4,  3,  4,  5,  2,  5,  7, 7,  6,  7,  8, 1,  4],
}
ridxs = []
for w, peg, r in zip(w_peg_r_iter6['w'], w_peg_r_iter6['peg'], w_peg_r_iter6['r']):
    ridxs.extend(df[(df['w'] == w) & (df['peg'] == peg) & (df['r'] == r)].index)
df = df.iloc[ridxs]
df.reset_index(inplace=True, drop=True)

In [ ]:
cols = ['Wo/w', 'qc/qd', 'rec', 'red', 'cac', 'cad', 'wec', 'wed']
score = {'di_rmse': [], 'di_r2': [], 'p': [], 'macro_p': [], 'r': [], 'macro_r': [], 'd_rmse': [], 'd_r2': [], 'p_rmse': [], 'p_r2': []}
di_models = []
o_models = []
d_models = []
p_models = []
for i in range(10):
    kfold = KFold(5, shuffle=True, random_state=i)
    for fold in range(5):
        train_idxs, test_idxs = list(kfold.split(df[cols], df['state']))[fold]
        train_df = df.iloc[train_idxs].copy()
        test_df = df.iloc[test_idxs].copy()
        train_x, test_x = train_df[cols], test_df[cols]
        train_y_di = (train_df['prev_d32'] / 0.6).to_numpy()
        test_y_di = (test_df['prev_d32'] / 0.6).to_numpy()
        di_model = RandomForestRegressor(random_state=12)
        di_model.fit(train_x, train_y_di)
        di_models.append(di_model)
        test_p_di = di_model.predict(test_x)
        score['di_rmse'].append(np.round(((test_p_di - test_y_di)**2).mean()**0.5, 4))
        score['di_r2'].append(np.round(r2_score(test_y_di, test_p_di), 4))
        train_df['di/Wi'] = di_model.predict(train_df[cols])
        test_df['di/Wi'] = di_model.predict(test_df[cols])

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_o = train_df['state'].to_numpy()
        test_y_o = test_df['state'].to_numpy()
        o_model = RandomForestClassifier(random_state=3)
        o_model.fit(train_x, train_y_o)
        o_models.append(o_model)
        test_p_o = o_model.predict(test_x)
        p_weight = 1 / 2 / np.array([(test_p_o == i).sum() for i in test_p_o])
        r_weight = 1 / 2 / np.array([(test_y_o == i).sum() for i in test_y_o])
        score['p'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['r'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['macro_p'].append(np.round(((test_p_o == test_y_o) * p_weight).sum(), 4))
        score['macro_r'].append(np.round(((test_p_o == test_y_o) * r_weight).sum(), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_d = (train_df['post_d32'] / (0.1 * train_df['w'])).to_numpy()
        test_y_d = (test_df['post_d32'] / (0.1 * test_df['w'])).to_numpy()
        d_model = RandomForestRegressor(random_state=12)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_p = train_df['post_d32_polydispersity'].to_numpy()
        test_y_p = test_df['post_d32_polydispersity'].to_numpy()
        p_model = RandomForestRegressor(random_state=12)
        p_model.fit(train_x, train_y_p)
        p_models.append(p_model)
        test_p_p = p_model.predict(test_x)
        score['p_rmse'].append(np.round(((test_p_p - test_y_p)**2).mean()**0.5, 4))
        score['p_r2'].append(np.round(r2_score(test_y_p, test_p_p), 4))

print('=============== di/Wi ================')
print('mean rmse: %.3f' % np.mean(score['di_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['di_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== occurrence ================')
print('mean p: %.3f' % np.mean(score['p']), end='\t\t')
print('mean macro_p: %.3f' % np.mean(score['macro_p']), end='\n')
print('mean r: %.3f' % np.mean(score['r']), end='\t\t')
print('mean macro_r: %.3f' % np.mean(score['macro_r']), end='\n')
print('===========================================', end='\n\n')

print('=============== do/Wo ================')
print('mean rmse: %.3f' % np.mean(score['d_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['d_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== sigma ================')
print('mean rmse: %.3f' % np.mean(score['p_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['p_r2']), end='\n')
print('======================================', end='\n\n')

=============== di/Wi ================
mean rmse: 0.144	mean r2: 0.858

=============== occurrence ================
mean p: 0.850		mean macro_p: 0.841
mean r: 0.850		mean macro_r: 0.852

=============== do/Wo ================
mean rmse: 0.058	mean r2: 0.940

=============== sigma ================
mean rmse: 4.250	mean r2: 0.025



In [ ]:
import itertools
qcs = [200, 400, 600, 800, 1000]
ws = [6, 10, 15]
pegs = [0, 5, 10, 15]
rs = [1, 2, 3, 4, 5, 6, 7, 8]
exp_w_peg_r_combos = list(zip(w_peg_r_iter6['w'], w_peg_r_iter6['peg'], w_peg_r_iter6['r']))
cand_df = {'w': [], 'peg': [], 'r': [], 'o_uncertainty': [], 'd_uncertainty': [], 'p_uncertainty': []}
for w, peg, r in itertools.product(ws, pegs, rs):
    if (w, peg, r) in exp_w_peg_r_combos: continue
    test_df = {'w': [], 'peg': [], 'r': [], 'qc': [], 'qd': []}
    for qc in qcs:
        test_df['w'].append(w)
        test_df['peg'].append(peg)
        test_df['r'].append(r)
        test_df['qc'].append(qc)
        test_df['qd'].append(qc / r)
    test_df = pd.DataFrame(test_df)
    test_df['uc'] = test_df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['ud'] = test_df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['momentumc'] = 998 * test_df['uc'] ** 2 / (1e-3 * 0.6)
    test_df['momentumd'] = 1.29 * test_df['ud'] ** 2 / (1e-3 * 0.6)
    test_df['viscosityc'] = (0.2563 * test_df['peg'] ** 1.7663 + 0.7890) * 1e-3 * test_df['uc'] / (1e-3 * 0.6) ** 2
    test_df['viscosityd'] = 18.6 * 1e-6 * test_df['ud'] / (1e-3 * 0.6) ** 2
    test_df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['Wo/w'] = test_df['w'] / 3
    test_df['qc/qd'] = test_df['qc'] / test_df['qd']
    test_df['rec'] = test_df['momentumc'] / test_df['viscosityc']
    test_df['red'] = test_df['momentumd'] / test_df['viscosityd']
    test_df['cac'] = test_df['viscosityc'] / test_df['interfacec']
    test_df['cad'] = test_df['viscosityd'] / test_df['interfaced']
    test_df['wec'] = test_df['momentumc'] / test_df['interfacec']
    test_df['wed'] = test_df['momentumd'] / test_df['interfaced']
    cand_df['w'].append(w)
    cand_df['peg'].append(peg)
    cand_df['r'].append(r)
    o_preds = []
    d_preds = []
    p_preds = []
    for i in range(len(di_models)):
        test_df['di/Wi'] = di_models[i].predict(test_df[cols])
        o_preds.append(o_models[i].predict(test_df[cols + ['di/Wi']]))
        d_preds.append(d_models[i].predict(test_df[cols + ['di/Wi']]))
        p_preds.append(p_models[i].predict(test_df[cols + ['di/Wi']]))
    cand_df['o_uncertainty'].append(np.stack(o_preds).std(axis=0).mean())
    cand_df['d_uncertainty'].append(np.stack(d_preds).std(axis=0).mean())
    cand_df['p_uncertainty'].append(np.stack(p_preds).std(axis=0).mean() / df['post_d32_polydispersity'].max())
cand_df = pd.DataFrame(cand_df)
cand_df['uncertainty'] = cand_df['o_uncertainty'] + cand_df['d_uncertainty'] + cand_df['p_uncertainty']
cand_df.sort_values('uncertainty', inplace=True, ascending=False)
print(cand_df['uncertainty'].mean())
cand_df.head(10)

0.12365792225791586


,w,peg,r,o_uncertainty,d_uncertainty,p_uncertainty,uncertainty
55,15,15,4,0.237980,0.008301,0.041287,0.287567
22,10,5,5,0.197980,0.022999,0.020339,0.241318
44,15,5,7,0.191652,0.018829,0.021509,0.231990
9,6,5,5,0.195959,0.012565,0.018365,0.226889
43,15,5,6,0.177980,0.017190,0.019827,0.214996
45,15,5,8,0.171652,0.019395,0.022127,0.213173
12,10,0,2,0.080000,0.031138,0.082729,0.193867
10,6,5,6,0.151652,0.013241,0.019273,0.184165
54,15,15,3,0.140000,0.006066,0.037267,0.183333
23,10,5,6,0.140000,0.022075,0.020445,0.182521


In [ ]:
# iteration 7
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_excel('./data/results.xlsx')
df['uc'] = df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['ud'] = df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['momentumc'] = 998 * df['uc'] ** 2 / (1e-3 * 0.6)
df['momentumd'] = 1.29 * df['ud'] ** 2 / (1e-3 * 0.6)
df['viscosityc'] = (0.2563 * df['peg'] ** 1.7663 + 0.7890) * 1e-3 * df['uc'] / (1e-3 * 0.6) ** 2
df['viscosityd'] = 18.6 * 1e-6 * df['ud'] / (1e-3 * 0.6) ** 2
df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
df['Wo/w'] = df['w'] / 3
df['qc/qd'] = df['qc'] / df['qd']
df['rec'] = df['momentumc'] / df['viscosityc']
df['red'] = df['momentumd'] / df['viscosityd']
df['cac'] = df['viscosityc'] / df['interfacec']
df['cad'] = df['viscosityd'] / df['interfaced']
df['wec'] = df['momentumc'] / df['interfacec']
df['wed'] = df['momentumd'] / df['interfaced']

w_peg_r_iter7 = {
    'w':   [6, 6, 10, 10, 10, 10, 15, 15, 15, 15, 10, 10, 6, 15, 15, 15, 6, 15, 15],
    'peg': [0, 5,  0,  5, 10, 15,  0,  5, 10, 15, 10, 10, 5,  0,  0,  0, 0,  5, 15],
    'r':   [2, 3,  1,  2,  3,  4,  3,  4,  5,  2,  5,  7, 7,  6,  7,  8, 1,  4,  4],
}
ridxs = []
for w, peg, r in zip(w_peg_r_iter7['w'], w_peg_r_iter7['peg'], w_peg_r_iter7['r']):
    ridxs.extend(df[(df['w'] == w) & (df['peg'] == peg) & (df['r'] == r)].index)
df = df.iloc[ridxs]
df.reset_index(inplace=True, drop=True)

In [ ]:
cols = ['Wo/w', 'qc/qd', 'rec', 'red', 'cac', 'cad', 'wec', 'wed']
score = {'di_rmse': [], 'di_r2': [], 'p': [], 'macro_p': [], 'r': [], 'macro_r': [], 'd_rmse': [], 'd_r2': [], 'p_rmse': [], 'p_r2': []}
di_models = []
o_models = []
d_models = []
p_models = []
for i in range(10):
    kfold = KFold(5, shuffle=True, random_state=i)
    for fold in range(5):
        train_idxs, test_idxs = list(kfold.split(df[cols], df['state']))[fold]
        train_df = df.iloc[train_idxs].copy()
        test_df = df.iloc[test_idxs].copy()
        train_x, test_x = train_df[cols], test_df[cols]
        train_y_di = (train_df['prev_d32'] / 0.6).to_numpy()
        test_y_di = (test_df['prev_d32'] / 0.6).to_numpy()
        di_model = RandomForestRegressor(random_state=12)
        di_model.fit(train_x, train_y_di)
        di_models.append(di_model)
        test_p_di = di_model.predict(test_x)
        score['di_rmse'].append(np.round(((test_p_di - test_y_di)**2).mean()**0.5, 4))
        score['di_r2'].append(np.round(r2_score(test_y_di, test_p_di), 4))
        train_df['di/Wi'] = di_model.predict(train_df[cols])
        test_df['di/Wi'] = di_model.predict(test_df[cols])

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_o = train_df['state'].to_numpy()
        test_y_o = test_df['state'].to_numpy()
        o_model = RandomForestClassifier(random_state=3)
        o_model.fit(train_x, train_y_o)
        o_models.append(o_model)
        test_p_o = o_model.predict(test_x)
        p_weight = 1 / 2 / np.array([(test_p_o == i).sum() for i in test_p_o])
        r_weight = 1 / 2 / np.array([(test_y_o == i).sum() for i in test_y_o])
        score['p'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['r'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['macro_p'].append(np.round(((test_p_o == test_y_o) * p_weight).sum(), 4))
        score['macro_r'].append(np.round(((test_p_o == test_y_o) * r_weight).sum(), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_d = (train_df['post_d32'] / (0.1 * train_df['w'])).to_numpy()
        test_y_d = (test_df['post_d32'] / (0.1 * test_df['w'])).to_numpy()
        d_model = RandomForestRegressor(random_state=12)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_p = train_df['post_d32_polydispersity'].to_numpy()
        test_y_p = test_df['post_d32_polydispersity'].to_numpy()
        p_model = RandomForestRegressor(random_state=12)
        p_model.fit(train_x, train_y_p)
        p_models.append(p_model)
        test_p_p = p_model.predict(test_x)
        score['p_rmse'].append(np.round(((test_p_p - test_y_p)**2).mean()**0.5, 4))
        score['p_r2'].append(np.round(r2_score(test_y_p, test_p_p), 4))

print('=============== di/Wi ================')
print('mean rmse: %.3f' % np.mean(score['di_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['di_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== occurrence ================')
print('mean p: %.3f' % np.mean(score['p']), end='\t\t')
print('mean macro_p: %.3f' % np.mean(score['macro_p']), end='\n')
print('mean r: %.3f' % np.mean(score['r']), end='\t\t')
print('mean macro_r: %.3f' % np.mean(score['macro_r']), end='\n')
print('===========================================', end='\n\n')

print('=============== do/Wo ================')
print('mean rmse: %.3f' % np.mean(score['d_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['d_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== sigma ================')
print('mean rmse: %.3f' % np.mean(score['p_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['p_r2']), end='\n')
print('======================================', end='\n\n')

=============== di/Wi ================
mean rmse: 0.134	mean r2: 0.929

=============== occurrence ================
mean p: 0.833		mean macro_p: 0.845
mean r: 0.833		mean macro_r: 0.810

=============== do/Wo ================
mean rmse: 0.054	mean r2: 0.965

=============== sigma ================
mean rmse: 3.041	mean r2: 0.617



In [ ]:
import itertools
qcs = [200, 400, 600, 800, 1000]
ws = [6, 10, 15]
pegs = [0, 5, 10, 15]
rs = [1, 2, 3, 4, 5, 6, 7, 8]
exp_w_peg_r_combos = list(zip(w_peg_r_iter7['w'], w_peg_r_iter7['peg'], w_peg_r_iter7['r']))
cand_df = {'w': [], 'peg': [], 'r': [], 'o_uncertainty': [], 'd_uncertainty': [], 'p_uncertainty': []}
for w, peg, r in itertools.product(ws, pegs, rs):
    if (w, peg, r) in exp_w_peg_r_combos: continue
    test_df = {'w': [], 'peg': [], 'r': [], 'qc': [], 'qd': []}
    for qc in qcs:
        test_df['w'].append(w)
        test_df['peg'].append(peg)
        test_df['r'].append(r)
        test_df['qc'].append(qc)
        test_df['qd'].append(qc / r)
    test_df = pd.DataFrame(test_df)
    test_df['uc'] = test_df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['ud'] = test_df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['momentumc'] = 998 * test_df['uc'] ** 2 / (1e-3 * 0.6)
    test_df['momentumd'] = 1.29 * test_df['ud'] ** 2 / (1e-3 * 0.6)
    test_df['viscosityc'] = (0.2563 * test_df['peg'] ** 1.7663 + 0.7890) * 1e-3 * test_df['uc'] / (1e-3 * 0.6) ** 2
    test_df['viscosityd'] = 18.6 * 1e-6 * test_df['ud'] / (1e-3 * 0.6) ** 2
    test_df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['Wo/w'] = test_df['w'] / 3
    test_df['qc/qd'] = test_df['qc'] / test_df['qd']
    test_df['rec'] = test_df['momentumc'] / test_df['viscosityc']
    test_df['red'] = test_df['momentumd'] / test_df['viscosityd']
    test_df['cac'] = test_df['viscosityc'] / test_df['interfacec']
    test_df['cad'] = test_df['viscosityd'] / test_df['interfaced']
    test_df['wec'] = test_df['momentumc'] / test_df['interfacec']
    test_df['wed'] = test_df['momentumd'] / test_df['interfaced']
    cand_df['w'].append(w)
    cand_df['peg'].append(peg)
    cand_df['r'].append(r)
    o_preds = []
    d_preds = []
    p_preds = []
    for i in range(len(di_models)):
        test_df['di/Wi'] = di_models[i].predict(test_df[cols])
        o_preds.append(o_models[i].predict(test_df[cols + ['di/Wi']]))
        d_preds.append(d_models[i].predict(test_df[cols + ['di/Wi']]))
        p_preds.append(p_models[i].predict(test_df[cols + ['di/Wi']]))
    cand_df['o_uncertainty'].append(np.stack(o_preds).std(axis=0).mean())
    cand_df['d_uncertainty'].append(np.stack(d_preds).std(axis=0).mean())
    cand_df['p_uncertainty'].append(np.stack(p_preds).std(axis=0).mean() / df['post_d32_polydispersity'].max())
cand_df = pd.DataFrame(cand_df)
cand_df['uncertainty'] = cand_df['o_uncertainty'] + cand_df['d_uncertainty'] + cand_df['p_uncertainty']
cand_df.sort_values('uncertainty', inplace=True, ascending=False)
print(cand_df['uncertainty'].mean())
cand_df.head(5)

0.12431713256899395


,w,peg,r,o_uncertainty,d_uncertainty,p_uncertainty,uncertainty
39,15,5,1,0.200000,0.033005,0.028695,0.261700
22,10,5,5,0.191652,0.028286,0.022317,0.242255
43,15,5,6,0.197980,0.022497,0.018231,0.238708
9,6,5,5,0.189631,0.026117,0.019135,0.234883
44,15,5,7,0.183303,0.022361,0.018126,0.223790
45,15,5,8,0.151652,0.023784,0.018250,0.193686
24,10,5,7,0.140000,0.028007,0.022818,0.190825
23,10,5,6,0.140000,0.027574,0.022215,0.189789
10,6,5,6,0.140000,0.027213,0.018328,0.185541
14,10,0,4,0.080000,0.042859,0.059948,0.182808


In [ ]:
# iteration 8
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_excel('./data/results.xlsx')
df['uc'] = df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['ud'] = df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['momentumc'] = 998 * df['uc'] ** 2 / (1e-3 * 0.6)
df['momentumd'] = 1.29 * df['ud'] ** 2 / (1e-3 * 0.6)
df['viscosityc'] = (0.2563 * df['peg'] ** 1.7663 + 0.7890) * 1e-3 * df['uc'] / (1e-3 * 0.6) ** 2
df['viscosityd'] = 18.6 * 1e-6 * df['ud'] / (1e-3 * 0.6) ** 2
df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
df['Wo/w'] = df['w'] / 3
df['qc/qd'] = df['qc'] / df['qd']
df['rec'] = df['momentumc'] / df['viscosityc']
df['red'] = df['momentumd'] / df['viscosityd']
df['cac'] = df['viscosityc'] / df['interfacec']
df['cad'] = df['viscosityd'] / df['interfaced']
df['wec'] = df['momentumc'] / df['interfacec']
df['wed'] = df['momentumd'] / df['interfaced']

w_peg_r_iter8 = {
    'w':   [6, 6, 10, 10, 10, 10, 15, 15, 15, 15, 10, 10, 6, 15, 15, 15, 6, 15, 15, 15],
    'peg': [0, 5,  0,  5, 10, 15,  0,  5, 10, 15, 10, 10, 5,  0,  0,  0, 0,  5, 15,  5],
    'r':   [2, 3,  1,  2,  3,  4,  3,  4,  5,  2,  5,  7, 7,  6,  7,  8, 1,  4,  4,  5],
}
ridxs = []
for w, peg, r in zip(w_peg_r_iter8['w'], w_peg_r_iter8['peg'], w_peg_r_iter8['r']):
    ridxs.extend(df[(df['w'] == w) & (df['peg'] == peg) & (df['r'] == r)].index)
df = df.iloc[ridxs]
df.reset_index(inplace=True, drop=True)

In [ ]:
cols = ['Wo/w', 'qc/qd', 'rec', 'red', 'cac', 'cad', 'wec', 'wed']
score = {'di_rmse': [], 'di_r2': [], 'p': [], 'macro_p': [], 'r': [], 'macro_r': [], 'd_rmse': [], 'd_r2': [], 'p_rmse': [], 'p_r2': []}
di_models = []
o_models = []
d_models = []
p_models = []
for i in range(10):
    kfold = KFold(5, shuffle=True, random_state=i)
    for fold in range(5):
        train_idxs, test_idxs = list(kfold.split(df[cols], df['state']))[fold]
        train_df = df.iloc[train_idxs].copy()
        test_df = df.iloc[test_idxs].copy()
        train_x, test_x = train_df[cols], test_df[cols]
        train_y_di = (train_df['prev_d32'] / 0.6).to_numpy()
        test_y_di = (test_df['prev_d32'] / 0.6).to_numpy()
        di_model = RandomForestRegressor(random_state=12)
        di_model.fit(train_x, train_y_di)
        di_models.append(di_model)
        test_p_di = di_model.predict(test_x)
        score['di_rmse'].append(np.round(((test_p_di - test_y_di)**2).mean()**0.5, 4))
        score['di_r2'].append(np.round(r2_score(test_y_di, test_p_di), 4))
        train_df['di/Wi'] = di_model.predict(train_df[cols])
        test_df['di/Wi'] = di_model.predict(test_df[cols])

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_o = train_df['state'].to_numpy()
        test_y_o = test_df['state'].to_numpy()
        o_model = RandomForestClassifier(random_state=15)
        o_model.fit(train_x, train_y_o)
        o_models.append(o_model)
        test_p_o = o_model.predict(test_x)
        p_weight = 1 / 2 / np.array([(test_p_o == i).sum() for i in test_p_o])
        r_weight = 1 / 2 / np.array([(test_y_o == i).sum() for i in test_y_o])
        score['p'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['r'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['macro_p'].append(np.round(((test_p_o == test_y_o) * p_weight).sum(), 4))
        score['macro_r'].append(np.round(((test_p_o == test_y_o) * r_weight).sum(), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_d = (train_df['post_d32'] / (0.1 * train_df['w'])).to_numpy()
        test_y_d = (test_df['post_d32'] / (0.1 * test_df['w'])).to_numpy()
        d_model = RandomForestRegressor(random_state=12)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_p = train_df['post_d32_polydispersity'].to_numpy()
        test_y_p = test_df['post_d32_polydispersity'].to_numpy()
        p_model = RandomForestRegressor(random_state=12)
        p_model.fit(train_x, train_y_p)
        p_models.append(p_model)
        test_p_p = p_model.predict(test_x)
        score['p_rmse'].append(np.round(((test_p_p - test_y_p)**2).mean()**0.5, 4))
        score['p_r2'].append(np.round(r2_score(test_y_p, test_p_p), 4))

print('=============== di/Wi ================')
print('mean rmse: %.3f' % np.mean(score['di_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['di_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== occurrence ================')
print('mean p: %.3f' % np.mean(score['p']), end='\t\t')
print('mean macro_p: %.3f' % np.mean(score['macro_p']), end='\n')
print('mean r: %.3f' % np.mean(score['r']), end='\t\t')
print('mean macro_r: %.3f' % np.mean(score['macro_r']), end='\n')
print('===========================================', end='\n\n')

print('=============== do/Wo ================')
print('mean rmse: %.3f' % np.mean(score['d_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['d_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== sigma ================')
print('mean rmse: %.3f' % np.mean(score['p_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['p_r2']), end='\n')
print('======================================', end='\n\n')

=============== di/Wi ================
mean rmse: 0.111	mean r2: 0.948

=============== occurrence ================
mean p: 0.891		mean macro_p: 0.893
mean r: 0.891		mean macro_r: 0.892

=============== do/Wo ================
mean rmse: 0.048	mean r2: 0.974

=============== sigma ================
mean rmse: 2.997	mean r2: 0.690



In [ ]:
import itertools
qcs = [200, 400, 600, 800, 1000]
ws = [6, 10, 15]
pegs = [0, 5, 10, 15]
rs = [1, 2, 3, 4, 5, 6, 7, 8]
exp_w_peg_r_combos = list(zip(w_peg_r_iter7['w'], w_peg_r_iter7['peg'], w_peg_r_iter7['r']))
cand_df = {'w': [], 'peg': [], 'r': [], 'o_uncertainty': [], 'd_uncertainty': [], 'p_uncertainty': []}
for w, peg, r in itertools.product(ws, pegs, rs):
    if (w, peg, r) in exp_w_peg_r_combos: continue
    test_df = {'w': [], 'peg': [], 'r': [], 'qc': [], 'qd': []}
    for qc in qcs:
        test_df['w'].append(w)
        test_df['peg'].append(peg)
        test_df['r'].append(r)
        test_df['qc'].append(qc)
        test_df['qd'].append(qc / r)
    test_df = pd.DataFrame(test_df)
    test_df['uc'] = test_df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['ud'] = test_df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
    test_df['momentumc'] = 998 * test_df['uc'] ** 2 / (1e-3 * 0.6)
    test_df['momentumd'] = 1.29 * test_df['ud'] ** 2 / (1e-3 * 0.6)
    test_df['viscosityc'] = (0.2563 * test_df['peg'] ** 1.7663 + 0.7890) * 1e-3 * test_df['uc'] / (1e-3 * 0.6) ** 2
    test_df['viscosityd'] = 18.6 * 1e-6 * test_df['ud'] / (1e-3 * 0.6) ** 2
    test_df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
    test_df['Wo/w'] = test_df['w'] / 3
    test_df['qc/qd'] = test_df['qc'] / test_df['qd']
    test_df['rec'] = test_df['momentumc'] / test_df['viscosityc']
    test_df['red'] = test_df['momentumd'] / test_df['viscosityd']
    test_df['cac'] = test_df['viscosityc'] / test_df['interfacec']
    test_df['cad'] = test_df['viscosityd'] / test_df['interfaced']
    test_df['wec'] = test_df['momentumc'] / test_df['interfacec']
    test_df['wed'] = test_df['momentumd'] / test_df['interfaced']
    cand_df['w'].append(w)
    cand_df['peg'].append(peg)
    cand_df['r'].append(r)
    o_preds = []
    d_preds = []
    p_preds = []
    for i in range(len(di_models)):
        test_df['di/Wi'] = di_models[i].predict(test_df[cols])
        o_preds.append(o_models[i].predict(test_df[cols + ['di/Wi']]))
        d_preds.append(d_models[i].predict(test_df[cols + ['di/Wi']]))
        p_preds.append(p_models[i].predict(test_df[cols + ['di/Wi']]))
    cand_df['o_uncertainty'].append(np.stack(o_preds).std(axis=0).mean())
    cand_df['d_uncertainty'].append(np.stack(d_preds).std(axis=0).mean())
    cand_df['p_uncertainty'].append(np.stack(p_preds).std(axis=0).mean() / df['post_d32_polydispersity'].max())
cand_df = pd.DataFrame(cand_df)
cand_df['uncertainty'] = cand_df['o_uncertainty'] + cand_df['d_uncertainty'] + cand_df['p_uncertainty']
cand_df.sort_values('uncertainty', inplace=True, ascending=False)
print(cand_df['uncertainty'].mean())
cand_df.head(5)

0.14846030562354412


,w,peg,r,o_uncertainty,d_uncertainty,p_uncertainty,uncertainty
10,6,5,6,0.287611,0.029026,0.017909,0.334545
23,10,5,6,0.260000,0.024849,0.017502,0.302351
45,15,5,8,0.257980,0.025246,0.017626,0.300852
24,10,5,7,0.257980,0.025071,0.017779,0.300830
11,6,5,8,0.251652,0.028818,0.017092,0.297562


In [ ]:
# iteration 9
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_excel('./data/results.xlsx')
df['uc'] = df['qc'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['ud'] = df['qd'] / 60 / 1e6 / (1e-3 * 0.6) ** 2
df['momentumc'] = 998 * df['uc'] ** 2 / (1e-3 * 0.6)
df['momentumd'] = 1.29 * df['ud'] ** 2 / (1e-3 * 0.6)
df['viscosityc'] = (0.2563 * df['peg'] ** 1.7663 + 0.7890) * 1e-3 * df['uc'] / (1e-3 * 0.6) ** 2
df['viscosityd'] = 18.6 * 1e-6 * df['ud'] / (1e-3 * 0.6) ** 2
df['interfacec'] = 0.035 / (1e-3 * 0.6) ** 2
df['interfaced'] = 0.035 / (1e-3 * 0.6) ** 2
df['Wo/w'] = df['w'] / 3
df['qc/qd'] = df['qc'] / df['qd']
df['rec'] = df['momentumc'] / df['viscosityc']
df['red'] = df['momentumd'] / df['viscosityd']
df['cac'] = df['viscosityc'] / df['interfacec']
df['cad'] = df['viscosityd'] / df['interfaced']
df['wec'] = df['momentumc'] / df['interfacec']
df['wed'] = df['momentumd'] / df['interfaced']

w_peg_r_iter9 = {
    'w':   [6, 6, 10, 10, 10, 10, 15, 15, 15, 15, 10, 10, 6, 15, 15, 15, 6, 15, 15, 15, 10],
    'peg': [0, 5,  0,  5, 10, 15,  0,  5, 10, 15, 10, 10, 5,  0,  0,  0, 0,  5, 15,  5,  5],
    'r':   [2, 3,  1,  2,  3,  4,  3,  4,  5,  2,  5,  7, 7,  6,  7,  8, 1,  4,  4,  5,  6],
}
ridxs = []
for w, peg, r in zip(w_peg_r_iter9['w'], w_peg_r_iter9['peg'], w_peg_r_iter9['r']):
    ridxs.extend(df[(df['w'] == w) & (df['peg'] == peg) & (df['r'] == r)].index)
df = df.iloc[ridxs]
df.reset_index(inplace=True, drop=True)

In [ ]:
cols = ['Wo/w', 'qc/qd', 'rec', 'red', 'cac', 'cad', 'wec', 'wed']
score = {'di_rmse': [], 'di_r2': [], 'p': [], 'macro_p': [], 'r': [], 'macro_r': [], 'd_rmse': [], 'd_r2': [], 'p_rmse': [], 'p_r2': []}
di_models = []
o_models = []
d_models = []
p_models = []
for i in range(10):
    kfold = KFold(5, shuffle=True, random_state=i)
    for fold in range(5):
        train_idxs, test_idxs = list(kfold.split(df[cols], df['state']))[fold]
        train_df = df.iloc[train_idxs].copy()
        test_df = df.iloc[test_idxs].copy()
        train_x, test_x = train_df[cols], test_df[cols]
        train_y_di = (train_df['prev_d32'] / 0.6).to_numpy()
        test_y_di = (test_df['prev_d32'] / 0.6).to_numpy()
        di_model = RandomForestRegressor(random_state=12)
        di_model.fit(train_x, train_y_di)
        di_models.append(di_model)
        test_p_di = di_model.predict(test_x)
        score['di_rmse'].append(np.round(((test_p_di - test_y_di)**2).mean()**0.5, 4))
        score['di_r2'].append(np.round(r2_score(test_y_di, test_p_di), 4))
        train_df['di/Wi'] = di_model.predict(train_df[cols])
        test_df['di/Wi'] = di_model.predict(test_df[cols])

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_o = train_df['state'].to_numpy()
        test_y_o = test_df['state'].to_numpy()
        o_model = RandomForestClassifier(random_state=12)
        o_model.fit(train_x, train_y_o)
        o_models.append(o_model)
        test_p_o = o_model.predict(test_x)
        p_weight = 1 / 2 / np.array([(test_p_o == i).sum() for i in test_p_o])
        r_weight = 1 / 2 / np.array([(test_y_o == i).sum() for i in test_y_o])
        score['p'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['r'].append(np.round((test_p_o == test_y_o).mean(), 4))
        score['macro_p'].append(np.round(((test_p_o == test_y_o) * p_weight).sum(), 4))
        score['macro_r'].append(np.round(((test_p_o == test_y_o) * r_weight).sum(), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_d = (train_df['post_d32'] / (0.1 * train_df['w'])).to_numpy()
        test_y_d = (test_df['post_d32'] / (0.1 * test_df['w'])).to_numpy()
        d_model = RandomForestRegressor(random_state=12)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols], test_df[cols]
        train_y_d = (train_df['post_d32'] / (train_df['di/Wi'] * 0.6)).to_numpy()
        test_y_d = (test_df['post_d32'] / (test_df['di/Wi'] * 0.6)).to_numpy()
        d_model = RandomForestRegressor(random_state=12)
        d_model.fit(train_x, train_y_d)
        d_models.append(d_model)
        test_p_d = d_model.predict(test_x)
        score['d_rmse'].append(np.round(((test_p_d - test_y_d)**2).mean()**0.5, 4))
        score['d_r2'].append(np.round(r2_score(test_y_d, test_p_d), 4))

        train_x, test_x = train_df[cols + ['di/Wi']], test_df[cols + ['di/Wi']]
        train_y_p = train_df['post_d32_polydispersity'].to_numpy()
        test_y_p = test_df['post_d32_polydispersity'].to_numpy()
        p_model = RandomForestRegressor(random_state=12)
        p_model.fit(train_x, train_y_p)
        p_models.append(p_model)
        test_p_p = p_model.predict(test_x)
        score['p_rmse'].append(np.round(((test_p_p - test_y_p)**2).mean()**0.5, 4))
        score['p_r2'].append(np.round(r2_score(test_y_p, test_p_p), 4))

print('=============== di/Wi ================')
print('mean rmse: %.3f' % np.mean(score['di_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['di_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== occurrence ================')
print('mean p: %.3f' % np.mean(score['p']), end='\t\t')
print('mean macro_p: %.3f' % np.mean(score['macro_p'][2:]), end='\n')
print('mean r: %.3f' % np.mean(score['r']), end='\t\t')
print('mean macro_r: %.3f' % np.mean(score['macro_r'][2:]), end='\n')
print('===========================================', end='\n\n')

print('=============== do/Wo ================')
print('mean rmse: %.3f' % np.mean(score['d_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['d_r2']), end='\n')
print('======================================', end='\n\n')

print('=============== sigma ================')
print('mean rmse: %.3f' % np.mean(score['p_rmse']), end='\t')
print('mean r2: %.3f' % np.mean(score['p_r2']), end='\n')
print('======================================', end='\n\n')